## Ali Albonaser
## Studentnr.1852275
# Monte Carlo simulatie treinen, Ankh-Morpork

Doel.
Ik lees het spoorboekje in, zet dit om naar losse treinen met geplande aankomst en vertrek, en simuleer vertragingen en het cascade-effect.

Wat ik rapporteer.
1. Gemiddeld aantal treinen per spoor dat 1, 5 en 10 minuten of meer te laat vertrekt.
2. Gemiddelde daadwerkelijke vertraging per bestemming, inclusief cascade-effect.
3. Spreiding per bestemming, dus de standaarddeviatie.

Belangrijke aannames.
Tijd is per minuut.
Als een trein vertraagd is, dan is de aankomstvertraging exponentieel verdeeld met het gegeven gemiddelde.
Een trein vertrekt niet eerder dan gepland.
Een vertraagde trein verkort zijn stop, maar staat minimaal de helft van de geplande stop op het perron.
Als een trein wil aankomen terwijl alle perrons bezet zijn, dan komt hij aan in de eerstvolgende minuut nadat een perron vrij is.
Als zijn geplande perron bezet is, mag hij een ander vrij perron gebruiken.

In [5]:
import math
import re
import numpy as np
import pandas as pd
import os

# data inlezen
BASE_DIR = os.getcwd()
CSV_PATH = os.path.join(BASE_DIR, "spoorboekje.csv")

df_raw = pd.read_csv(CSV_PATH)
df_raw.head()


,Unnamed: 0,Tijd,Spoor 1,Spoor 2,Spoor 3,Spoor 4
0,0,05:05:00,NaN,Aankomst Sto Plains 1,NaN,NaN
1,1,05:08:00,NaN,Vertrek Sto Plains 1,NaN,NaN
2,2,05:15:00,NaN,Aankomst Sto Plains 2,NaN,NaN
3,3,05:18:00,NaN,Vertrek Sto Plains 2,NaN,NaN
4,4,05:25:00,NaN,Aankomst Sto Plains 3,NaN,NaN


## 1. Spoorboekje omzetten naar treinen

In het CSV-bestand staat per minuut in elke spoor-kolom een tekst zoals:
Aankomst Sto Plains 1 of Vertrek Sto Plains 1.

Ik maak hier een tabel van met per trein:
bestemming, trein_nummer, gepland_aankomst, gepland_vertrek, gepland_spoor, geplande_stop, minimale_stop.

In [6]:
def time_to_min(tstr):
    # Verwacht HH:MM:SS
    h, m, s = [int(x) for x in tstr.split(":")]
    return 60*h + m

EVENT_RE = re.compile(r"^(Aankomst|Vertrek)\s+(.+?)\s+(\d+)$")

def parse_trains(df):
    df = df.copy()
    # Soms staat er een extra index kolom
    if "Unnamed: 0" in df.columns:
        df = df.drop(columns=["Unnamed: 0"])
    df["t_min"] = df["Tijd"].apply(time_to_min)

    records = []
    spoor_cols = [c for c in df.columns if c.startswith("Spoor")]

    # tijdelijk opslaan: (spoor, bestemming, nr) -> aankomsttijd
    arrival_lookup = {}

    for spoor in spoor_cols:
        for _, row in df.iterrows():
            txt = row[spoor]
            if pd.isna(txt):
                continue
            txt = str(txt).strip()
            m = EVENT_RE.match(txt)
            if not m:
                continue
            typ, dest, nr = m.group(1), m.group(2), int(m.group(3))
            key = (spoor, dest, nr)

            if typ == "Aankomst":
                arrival_lookup[key] = int(row["t_min"])
            else:  # Vertrek
                if key not in arrival_lookup:
                    # In theorie kan dit niet, maar we vangen het af
                    continue
                arr = arrival_lookup.pop(key)
                dep = int(row["t_min"])
                dwell = dep - arr
                min_dwell = int(math.ceil(dwell / 2))
                records.append({
                    "dest": dest,
                    "train_nr": nr,
                    "spoor_planned": spoor,
                    "arr_planned": arr,
                    "dep_planned": dep,
                    "dwell_planned": dwell,
                    "min_dwell": min_dwell
                })

    trains = pd.DataFrame(records)
    trains = trains.sort_values(["arr_planned", "spoor_planned", "dest", "train_nr"]).reset_index(drop=True)
    return trains

trains = parse_trains(df_raw)
trains.head(), trains.shape

(          dest  train_nr spoor_planned  arr_planned  dep_planned  \
 0   Sto Plains         1       Spoor 2          305          308   
 1   Sto Plains         2       Spoor 2          315          318   
 2   Sto Plains         3       Spoor 2          325          328   
 3   Sto Plains         4       Spoor 2          335          338   
 4  Pseudopolis         1       Spoor 4          337          340   
 
    dwell_planned  min_dwell  
 0              3          2  
 1              3          2  
 2              3          2  
 3              3          2  
 4              3          2  ,
 (339, 7))

## 2. Vertragingstabel

Kans op vertraging is Bernoulli.
Als vertraagd, dan is de aankomstvertraging exponentieel verdeeld.
Gemiddelde van de exponentiële verdeling is de waarde in minuten uit de tabel.

In [7]:
delay_params = {
    "Sto Plains":   {"p": 0.3, "mean": 8},
    "Pseudopolis":  {"p": 0.5, "mean": 5},
    "Lancre":       {"p": 0.3, "mean": 3},
    "Quirm":        {"p": 0.2, "mean": 15},
    "Genoa":        {"p": 0.2, "mean": 20},
    "Brindisi":     {"p": 0.1, "mean": 11},
    "Llamedos":     {"p": 0.4, "mean": 7},
    "Omnia":        {"p": 0.3, "mean": 11},
    "Borogravia":   {"p": 0.2, "mean": 8},
    "Klatch":       {"p": 0.5, "mean": 2},
    "Uberwald":     {"p": 0.2, "mean": 14},
    "Tsort":        {"p": 0.8, "mean": 6},
    "Ephebe":       {"p": 0.4, "mean": 9},
}

# Controle: alle bestemmingen uit het spoorboekje moeten in delay_params zitten
missing = sorted(set(trains["dest"]) - set(delay_params.keys()))
missing

[]

Als je hierboven een lijst ziet, dan staat er een bestemming in het spoorboekje die niet in de tabel zit. Dan moet je die toevoegen.

## 3. Eigen RNG

Ik gebruik een eigen RNG op basis van PCG32.
Hiermee maak ik uniforme getallen in [0,1), en daarvan maak ik Bernoulli en exponentieel.

In [8]:
class PCG32:
    # PCG32: goede kwaliteit, compact, snel
    def __init__(self, seed=42, seq=54):
        self.state = 0
        self.inc = (seq << 1) | 1
        self.seed(seed)

    def seed(self, seed):
        self.state = 0
        self.next_uint32()
        self.state = (self.state + seed) & 0xFFFFFFFFFFFFFFFF
        self.next_uint32()

    def next_uint32(self):
        oldstate = self.state
        self.state = (oldstate * 6364136223846793005 + self.inc) & 0xFFFFFFFFFFFFFFFF
        xorshifted = ((oldstate >> 18) ^ oldstate) >> 27
        rot = oldstate >> 59
        return ((xorshifted >> rot) | (xorshifted << ((-rot) & 31))) & 0xFFFFFFFF

    def random(self):
        # uniform in [0,1)
        return self.next_uint32() / 2**32

    def bernoulli(self, p):
        return 1 if self.random() < p else 0

    def exponential(self, mean):
        # inverse transform: -mean * ln(1-u)
        u = self.random()
        if u == 1.0:
            u = (2**32 - 1) / 2**32
        return -mean * math.log(1.0 - u)

# Snelle test
rng = PCG32(seed=123)
[rng.random() for _ in range(3)]

[0.641917810542509, 0.47173519362695515, 0.949481796938926]

## 4. Eén simulatie van een dag

Werkwijze per trein.
1. Trek aankomstvertraging op basis van bestemming.
2. Bepaal de gewenste aankomsttijd.
3. Kies een spoor.
Eerst het geplande spoor als dat vrij is.
Anders een ander vrij spoor.
Als niets vrij is, wacht tot het eerste spoor vrij is, en kom aan in de minuut erna.
4. Bepaal vertrektijd met het halve-stop-regel.
Vertrek = max(gepland vertrek, aankomst + minimale stop).
5. Zet het spoor bezet tot vertrektijd.

Uitkomst per trein.
aankomst_actueel, vertrek_actueel, gekozen_spoor, vertrekvertraging.

In [10]:
SPOREN = ["Spoor 1", "Spoor 2", "Spoor 3", "Spoor 4"]

def simulate_day(trains_df, rng: PCG32):
    # per spoor: tot wanneer bezet
    occupied_until = {sp: -10**9 for sp in SPOREN}

    out = trains_df.copy()

    arr_actual = np.zeros(len(out), dtype=int)
    dep_actual = np.zeros(len(out), dtype=int)
    spoor_used = np.empty(len(out), dtype=object)

    for i, row in out.iterrows():
        dest = row["dest"]
        p = delay_params[dest]["p"]
        mean = delay_params[dest]["mean"]

        # aankomstvertraging
        delay = 0.0
        if rng.bernoulli(p) == 1:
            delay = rng.exponential(mean)

        # afronden naar minuten, naar boven zodat je niet 'gratis' tijd wint
        delay_min = int(math.ceil(delay))
        arr0 = int(row["arr_planned"]) + delay_min

        # spoor kiezen
        planned_spoor = row["spoor_planned"]
        chosen = None
        arr = arr0

        # als gepland spoor vrij is op aankomsttijd
        if occupied_until[planned_spoor] < arr:
            chosen = planned_spoor
        else:
            # check andere vrije sporen
            free_sporen = [sp for sp in SPOREN if occupied_until[sp] < arr]
            if len(free_sporen) > 0:
                chosen = free_sporen[0]
            else:
                # wachten tot eerste spoor vrij komt
                earliest_spoor = min(SPOREN, key=lambda sp: occupied_until[sp])
                arr = occupied_until[earliest_spoor] + 1
                chosen = earliest_spoor

        # vertrek met minimale stop en niet eerder dan gepland vertrek
        min_dwell = int(row["min_dwell"])
        dep = max(int(row["dep_planned"]), arr + min_dwell)

        # spoor bezetten
        occupied_until[chosen] = dep

        arr_actual[i] = arr
        dep_actual[i] = dep
        spoor_used[i] = chosen

    out["arr_actual"] = arr_actual
    out["dep_actual"] = dep_actual
    out["spoor_used"] = spoor_used
    out["dep_delay"] = out["dep_actual"] - out["dep_planned"]
    return out

# 1 run demo
demo = simulate_day(trains, PCG32(seed=1))
demo.head()

,dest,train_nr,spoor_planned,arr_planned,dep_planned,dwell_planned,min_dwell,arr_actual,dep_actual,spoor_used,dep_delay
0,Sto Plains,1,Spoor 2,305,308,3,2,305,308,Spoor 2,0
1,Sto Plains,2,Spoor 2,315,318,3,2,329,331,Spoor 2,13
2,Sto Plains,3,Spoor 2,325,328,3,2,325,328,Spoor 1,0
3,Sto Plains,4,Spoor 2,335,338,3,2,335,338,Spoor 2,0
4,Pseudopolis,1,Spoor 4,337,340,3,2,343,345,Spoor 4,5


## 5. Monte Carlo: 1000 runs

Ik herhaal een dag minimaal 1000 keer.
Daarna bereken ik:
a. Gemiddelde aantallen te late vertrekken per spoor, voor drempels 1, 5, 10.
b. Gemiddelde vertrekvertraging per bestemming.
c. Standaarddeviatie per bestemming.

In [11]:
def mc_simulate(trains_df, n_runs=1000, seed=42):
    # Opslag voor spoor-tellingen per run
    thresholds = [1, 5, 10]
    spoor_counts = {sp: {thr: [] for thr in thresholds} for sp in SPOREN}

    # Opslag voor bestemming vertragingen over alle runs
    dest_delays = {d: [] for d in delay_params.keys()}

    rng = PCG32(seed=seed, seq=54)

    for r in range(n_runs):
        day = simulate_day(trains_df, rng)

        # per spoor: tel hoeveel dep_delay >= drempel
        for sp in SPOREN:
            mask = (day["spoor_used"] == sp)
            delays = day.loc[mask, "dep_delay"].to_numpy()
            for thr in thresholds:
                spoor_counts[sp][thr].append(int(np.sum(delays >= thr)))

        # per bestemming: alle vertrekvertragingen verzamelen
        for dest, grp in day.groupby("dest"):
            dest_delays[dest].extend(grp["dep_delay"].to_list())

    # Resultaat 1: gemiddelde counts per spoor
    spoor_rows = []
    for sp in SPOREN:
        row = {"spoor": sp}
        for thr in thresholds:
            row[f"gem_count_delay_ge_{thr}"] = float(np.mean(spoor_counts[sp][thr]))
        spoor_rows.append(row)
    spoor_summary = pd.DataFrame(spoor_rows)

    # Resultaat 2 + 3: mean en std per bestemming
    dest_rows = []
    for dest, vals in dest_delays.items():
        vals = np.array(vals, dtype=float)
        dest_rows.append({
            "bestemming": dest,
            "gem_dep_delay": float(np.mean(vals)),
            "std_dep_delay": float(np.std(vals, ddof=1)),
        })
    dest_summary = pd.DataFrame(dest_rows).sort_values("gem_dep_delay", ascending=False).reset_index(drop=True)

    return spoor_summary, dest_summary

spoor_summary, dest_summary = mc_simulate(trains, n_runs=1000, seed=123)

spoor_summary, dest_summary.head(10)

(     spoor  gem_count_delay_ge_1  gem_count_delay_ge_5  gem_count_delay_ge_10
 0  Spoor 1                14.805                 8.115                  4.757
 1  Spoor 2                27.661                16.202                  8.984
 2  Spoor 3                 8.827                 4.467                  2.300
 3  Spoor 4                31.185                14.486                  6.778,
     bestemming  gem_dep_delay  std_dep_delay
 0        Genoa       3.224500      11.542597
 1        Quirm       2.786405       8.797006
 2        Tsort       2.493200       5.095718
 3  Pseudopolis       2.454865       4.581324
 4   Sto Plains       2.291907       5.776918
 5     Llamedos       1.972529       5.073089
 6        Omnia       1.762400       6.347819
 7     Uberwald       1.748500       7.250193
 8       Ephebe       1.591600       5.156018
 9       Lancre       0.873436       2.195155)

## 6. Resultaten netjes tonen

In [12]:
# Maak het leesbaar
spoor_summary_style = spoor_summary.copy()
spoor_summary_style

,spoor,gem_count_delay_ge_1,gem_count_delay_ge_5,gem_count_delay_ge_10
0,Spoor 1,14.805,8.115,4.757
1,Spoor 2,27.661,16.202,8.984
2,Spoor 3,8.827,4.467,2.300
3,Spoor 4,31.185,14.486,6.778


In [13]:
dest_summary

,bestemming,gem_dep_delay,std_dep_delay
0,Genoa,3.224500,11.542597
1,Quirm,2.786405,8.797006
2,Tsort,2.493200,5.095718
3,Pseudopolis,2.454865,4.581324
4,Sto Plains,2.291907,5.776918
5,Llamedos,1.972529,5.073089
6,Omnia,1.762400,6.347819
7,Uberwald,1.748500,7.250193
8,Ephebe,1.591600,5.156018
9,Lancre,0.873436,2.195155


## 7. Conclusie

Uit de Monte Carlo simulatie blijkt dat het cascade-effect een duidelijke invloed heeft op de doorstroming. Spoor 4 en Spoor 2 hebben gemiddeld de meeste vertraagde vertrekken, terwijl Spoor 3 het minst gevoelig is voor vertragingen. Dit laat zien dat sommige sporen structureel zwaarder belast zijn dan andere.

Wat betreft de bestemmingen heeft Genoa de hoogste gemiddelde vertrekvertraging en ook de grootste spreiding. Dit betekent dat vertragingen daar niet alleen groter zijn, maar ook sterk kunnen variëren. Bestemmingen zoals Lancre hebben zowel een lage gemiddelde vertraging als een kleine spreiding en zijn daardoor stabieler.

De resultaten laten zien dat aanpassingen aan de infrastructuur vooral effect zullen hebben op de sporen en bestemmingen waar vertragingen zich het meest opstapelen.